In [41]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

import boto3
import re
from sagemaker import get_execution_role
import sagemaker

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [42]:
endpoint_name = 'xgboost-v1'
predictor = sagemaker.predictor.Predictor (endpoint_name=endpoint_name)

In [43]:
predictor.serializer = CSVSerializer()

In [44]:
df_all = pd.read_csv('test.csv')

In [45]:
df_all=df_all[["EffectiveDate","Location","Instance Type","Operating System","day","month","year","PricePerUnit"]]

In [46]:
arr_test = df_all[df_all.columns[1:-1]].values

In [47]:
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    result = result.decode("utf-8")
    result = result.split(',')
    print (arr.shape)
    predictions += [float(r) for r in result]

(130002, 6)
(130002, 6)
(130002, 6)
(130002, 6)
(130002, 6)
(130002, 6)
(130002, 6)
(130002, 6)
(130002, 6)
(130002, 6)


In [48]:
len(predictions)

1300020

In [49]:
np.expm1(predictions)

array([12.95024531, 12.95024531, 12.95024531, ..., 62.28985864,
       62.28985864, 62.28985864])

In [50]:
df_all['PricePerUnit'] = np.expm1(predictions)

In [51]:
df_all[["Location","Instance Type","Operating System","day","month","year","PricePerUnit"]].to_csv('predicted.csv',index=False)